Приветсвую тех, кто заглнянул сюда, в мой ноутбук!
Как вы наверное поняли из названия моего репозитория/датасета с Kaggle это ноутбук, посвященный анализу стендов из аниме "Невероятнве приключения ДжоДжо"

Сначала немного контекста:

Стенд -- это физическое проявление "жизненной энергии" человека, его сильный дух

У стендов есть характеристики:

- Разрушительная сила: Измеряет силу стенда и его способность вызвать разрушения (физические повреждения или сопутствующий ущерб окружающей среде) за определенный период времени.

- Скорость: Измеряет ловкость и рефлексы стенда, а также скорость выполнения.

- Радиус: Измеряет компромисс диапазона проявления стенда, диапазон влияния способностей и пространственную мобильность.

- Выносливость: Измеряет продолжительность времени, в течение которого стенд может активно поддерживать свою способность. Стенды дальней дистанции, способные работать удаленно от своих пользователей, такие как Lovers и Empress, описываются как обладающие высокой выносливостью. Это также относится к материализованным стендам, такие как Yellow Temperance и Atom Heart Father, которые могут привязываться к объектам на длительное время.

- Точность: Измеряет точность стенда и радиус воздействия/эффекта его способностей на указанные цели. Стенды автоматического типа обычно оцениваются по рангу D или ниже, за некоторыми исключениями.

- Потенциал развития: Измеряет возможные функции стенда, использование его способностей и сил, а также способность улучшить его общие возможности. Ранг уменьшается по мере того, как пользователь осваивает свой стенд.

Пример описания характеристики стенда:

<div style="text-align:center"> 

<img src="https://pm1.aminoapps.com/6955/8540d06f69c1111b7bae390fb9b66effe3809d46r1-1080-1017v2_hq.jpg" alt="drawing" width="200"/>

</div>

Зная это, приступим к анализу данного датасета и найдем что-нибудь интересное!

<div style="text-align:center"> 

![Ой, Джоске](https://avatars.dzeninfra.ru/get-zen_doc/3680683/pub_62dfb5f901cd0904a73542c1_62dfbe04c58eaf42ac4bd2fe/orig)


# 1 часть -- Импорт библиотек

In [304]:
import pandas as pd
import numpy as np

import scipy.stats as sts
import matplotlib.pyplot as plt
import plotly.express as px
 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2 часть -- Начало анализа, первый взгялд на данные

In [305]:
begin_dataset = pd.read_csv('JoJo_Bizarre_Adventure_All_Stands_Stats.csv')

# Выведем 10 случайных строк, чтобы посмотреть на датасет
display(begin_dataset.sample(10))

display(begin_dataset.info())


,Stand,PWR,SPD,RNG,STA,PRC,DEV,Story
29,The World,A,A,C,A,B,B,Part 3: Stardust Crusaders
37,Boy II Man,C,B,B,A,C,C,Part 4: Diamond is Unbreakable
117,Ball Breaker,A,A,D,B,C,B,Part 7: Steel Ball Run
18,Hanged Man,C,A,A,B,D,D,Part 3: Stardust Crusaders
63,Gold Experience,C,A,C,D,C,A,Part 5: Vento Aureo
41,Enigma,E,E,C,A,C,C,Part 4: Diamond is Unbreakable
0,Anubis,B,B,E,A,E,C,Part 3: Stardust Crusaders
94,Goo Goo Dolls,D,C,B,D,B,B,Part 6: Stone Ocean
132,Ticket to Ride,E,E,E,C,E,C,Part 7: Steel Ball Run
59,Echoes (ACT3),B,B,C,B,C,A,Part 4: Diamond is Unbreakable


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Stand   156 non-null    object
 1   PWR     156 non-null    object
 2   SPD     156 non-null    object
 3   RNG     156 non-null    object
 4   STA     156 non-null    object
 5   PRC     156 non-null    object
 6   DEV     156 non-null    object
 7   Story   156 non-null    object
dtypes: object(8)
memory usage: 9.9+ KB


None

# 2.1 -- Части ДжоДжо

Сначала я хочу переимновать колонки статистики для удобства, а также очень хочу посмотреть, сколько у нас есть частей всего (сколько уникальных значений в колонке Story)

In [306]:
# Переименуем колнки
dict_to_replace = {'PWR': 'Сила', 
                   'SPD': 'Скорость',
                   'RNG': 'Дальность',
                   'STA': 'Выносливость',
                   'PRC': 'Точность',
                   'DEV': 'Потенциал'}

begin_dataset.rename(columns=dict_to_replace, inplace=True)

display(begin_dataset)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
0,Anubis,B,B,E,A,E,C,Part 3: Stardust Crusaders
1,Atum,D,C,D,B,D,D,Part 3: Stardust Crusaders
2,Bastet,E,E,B,A,E,E,Part 3: Stardust Crusaders
3,Cream,B,B,D,C,C,D,Part 3: Stardust Crusaders
4,Death Thirteen,C,C,E,B,D,B,Part 3: Stardust Crusaders
...,...,...,...,...,...,...,...,...
151,Nightbird Flying,E,A,A,A,E,E,Light Novel: Purple Haze Feedback
152,Purple Haze Distortion,A,B,E,E,C,unknown,Light Novel: Purple Haze Feedback
153,Rainy Day Dream Away,E,B,A,A,E,E,Light Novel: Purple Haze Feedback
154,Remote Romance,E,D,infinite,B,C,E,Featured


Теперь посмотрим сначала на то, какие у нас есть части данного аниме, а также посмотрим на распределение характеристик, чтобы понять общую картину

In [307]:
parts_of_jojo = begin_dataset.value_counts('Story')
print(parts_of_jojo)

Story
Part 3: Stardust Crusaders           33
Part 5: Vento Aureo                  29
Part 4: Diamond is Unbreakable       28
Part 6: Stone Ocean                  26
Part 7: Steel Ball Run               24
Light Novel: Purple Haze Feedback     7
Part 8: JoJolion                      7
Featured                              2
Name: count, dtype: int64


Что ж, по крайней мере, я дошел до Stone Ocean, так что я думаю, что большинство стендов мне будет знакомо :)

Большинство стендов находятся в 3 части, это круто, так как нас только знакомили с ними в этом сезоне

Меньше всего стендов находятся в "Featured", честно, сам не знаю, что это такое, так что в теории можно вывести эти стенды и посмотреть на них получше, может я что-нибудь да вспомню

## 2.2 -- Стенды категории "Featured"

In [308]:
featured_part = begin_dataset[begin_dataset['Story'] == 'Featured']

display(featured_part)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
154,Remote Romance,E,D,infinite,B,C,E,Featured
155,House of Holy,E,B,C,A,D,D,Featured


Я узнал про эти стенды подробнее и оказывается, что **Remote Romance** -- это стенд, разработанный Хирохико Араки (создатель ДжоДжо) для выставки JoJo 2012. Его целью было продвижение художественной выставки через Интернет в прямом эфире.


<div style="text-align:center"> 

<img src="https://static.wikia.nocookie.net/jjba/images/3/32/Remote_Romance.png/revision/latest?cb=20170309021209" alt="drawing" width="200"/>

</div>

Подробнее про **Remote Romance** можете почитать [здесь](https://jojo.fandom.com/ru/wiki/Remote_Romance)

Так что этого стенда и не было в аниме/манге


Теперь узнаем про **House of Holy**. С ним очень интересно все -- это стенд-отель-лабиринт и у него нет внешнего вида, его опасность заключается в том, что любой, кто войдет в отель -- сразу окажется в лабиринте и в течение часа надо найти выход, если человек не найдет выход, то он пропадет в лабиринте навсегда. Любой физический урон неэффективен. 

[Подробнее про сам стенд](https://jojo.fandom.com/ru/wiki/House_of_Holy)

Также он фигурировал в специальной квест-игре, где участники в Японии должны решить загадки владельца стенда и выбраться из лабиринта

[Подробнее про квест-игру](https://jojo.fandom.com/ru/wiki/JoJo%27s_Bizarre_Escape:_The_Hotel)

Можно предположить, что этот стенд появляется в 3 части и его владелец является приспешником Дио (о нем позже поговорим)

## 2.3 -- Общее распределение характеристик

### 2.3.1 -- Сила

Теперь рассмотрим общее распределение характреристик стендов и поймем, стендов каких типов по качеству характеристик (A-E) больше всего

In [309]:
power_stats = begin_dataset['Сила'].value_counts()

display(power_stats)

Сила
E            35
A            32
C            27
B            26
D            17
undefined    16
unknown       2
infinite      1
Name: count, dtype: int64

Давайте, чтобы не путаться, мы будем обозначать характеристики цифрами (от 1 до 5), соответственно, это у нас будет (E-A)

In [310]:
replace_stat_dict = {'A': 5,
                     'B': 4,
                     'C': 3,
                     'D': 2,
                     'E': 1}

dataset_columns = begin_dataset.columns[1:-1]

for i in dataset_columns:
    begin_dataset[i].replace(replace_stat_dict, inplace = True)

display(begin_dataset)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders
...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback
154,Remote Romance,1,2,infinite,4,3,1,Featured


Посмотрим еще раз

In [311]:
power_stats = begin_dataset['Сила'].value_counts()

display(power_stats)

Сила
1            35
5            32
3            27
4            26
2            17
undefined    16
unknown       2
infinite      1
Name: count, dtype: int64

Как видно, у нас большинство стендов -- слабые по силе *(1)*, но также много и очень сильных *(5)*


Также у нас есть такие параметры, как неопределено, неизвестно и бесконечно

**Бесконечно** понятно, бесконечная мощь, непобедимый по силе, у нас такой только 1 стенд, давайте посмотрим, кто это может быть


In [312]:
inf_power_stand = begin_dataset[begin_dataset['Сила'] == 'infinite']

display(inf_power_stand)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo


А, ну тогда понятно, да, действительно, у нас только один **Gold Experience Requiem** (GER) настолько сильный :)

Бесконечное все -- непобедимый стенд, действительно

<p align="center">
    <img src="https://steamuserimages-a.akamaihd.net/ugc/775103339591880851/0452250037714568022DC0681A3AD2704C721D54/?imw=5000&amp;imh=5000&amp;ima=fit&amp;impolicy=Letterbox&amp;imcolor=%23000000&amp;letterbox=false" alt="Gold Experience Requiem 1" width="300">
</p>

<p align="center">
    <img src="https://i.pinimg.com/originals/6a/cc/c6/6accc6d646fb378c932fcd2b73e16307.png" alt="Gold Experience Requiem 2" width="300">
</p>

<p align="center">
    <img src="https://steamuserimages-a.akamaihd.net/ugc/5108802431775556322/46F42EC084464AA428E7703388BA664F0A560A9C/?imw=512&amp;imh=287&amp;ima=fit&amp;impolicy=Letterbox&amp;imcolor=%23000000&amp;letterbox=true" alt="Gold Experience Requiem 3" width="300">
</p>


Рассмотрим теперь стенды с пометкой *undefined* и *unknown* и решим, они все таки сильные или слабые

In [313]:
unknown_stands = begin_dataset[
    begin_dataset['Сила'] == 'unknown'
]

undefined_stands = begin_dataset[
    begin_dataset['Сила'] == 'undefined'
]

display(unknown_stands)
display(undefined_stands)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
90,Whitesnake,unknown,2,unknown,5,unknown,unknown,Part 6: Stone Ocean
104,"Green, Green Grass of Home",unknown,unknown,unknown,unknown,unknown,unknown,Part 6: Stone Ocean


,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story
85,Rolling Stones,undefined,4,5,5,1,undefined,Part 5: Vento Aureo
91,C-Moon,undefined,4,4,unknown,unknown,unknown,Part 6: Stone Ocean
92,Jail House Lock,undefined,3,4,5,undefined,undefined,Part 6: Stone Ocean
95,Under World,undefined,3,5,3,undefined,undefined,Part 6: Stone Ocean
96,Sky High,undefined,undefined,4,3,undefined,undefined,Part 6: Stone Ocean
99,Bohemian Rhapsody,undefined,undefined,infinite,5,undefined,undefined,Part 6: Stone Ocean
100,Burning Down the House,undefined,undefined,undefined,undefined,undefined,undefined,Part 6: Stone Ocean
102,Dragon's Dream,undefined,undefined,undefined,5,undefined,undefined,Part 6: Stone Ocean
108,Limp Bizkit,undefined,4,4,5,3,1,Part 6: Stone Ocean
116,20th Century Boy,undefined,3,undefined,5,2,3,Part 7: Steel Ball Run


Ну... В теории можно построить график распределения и +- случайно распределить параметры, на основе существующих данных. Допустим, мы присвоим 1 с такой вероятностью, 2 с другой и т.д

Алгоритм примерно следующий:

находим сколько стендов имеют силу 1, например, и делим на общее количество стендов

In [314]:
done_values_power = begin_dataset[
    (begin_dataset['Сила'] != 'undefined') & (begin_dataset['Сила'] != 'unknown') & (begin_dataset['Сила'] != 'infinite')
]

power_stats_histogram = px.histogram(done_values_power['Сила'], nbins = 8)
power_stats_histogram.show()

In [315]:
additional_done_values_power = done_values_power.value_counts('Сила').reset_index().rename(columns = {'count': 'Кол-во'})

additional_done_values_power['probability'] = additional_done_values_power['Кол-во'] / additional_done_values_power['Кол-во'].sum()

display(additional_done_values_power)

,Сила,Кол-во,probability
0,1,35,0.255474
1,5,32,0.233577
2,3,27,0.197080
3,4,26,0.189781
4,2,17,0.124088


Теперь, имея данные вероятности, мы можем заполнить наши значения

In [328]:
stands_to_be_replaced = begin_dataset[
    (begin_dataset['Сила'] == 'unknown') | (begin_dataset['Сила'] == 'undefined')
]

def replace_empty_power(row):
    row['Сила'] = np.random.choice(additional_done_values_power['Сила'], p = additional_done_values_power['probability'])
    return row['Сила']

stands_to_be_replaced['Сила'] = stands_to_be_replaced.apply(replace_empty_power, axis = 1)

begin_dataset.loc[stands_to_be_replaced.index, 'Сила'] = stands_to_be_replaced['Сила']

display(begin_dataset)

display(begin_dataset.value_counts('Сила').reset_index())


,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд
...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд


,Сила,count
0,1,41
1,5,36
2,3,31
3,4,29
4,2,18
5,infinite,1


Предположим, что стенды с силой 3 и больше, а также таинственный стенд, у которого сила бесконечна -- сильные, остальные -- слабые, тогда давайте отобразим их распределение на круговой диаграмме (как раз будут хорошие категории)

In [317]:
def assign_strongest_stands(row):
    if row['Сила'] == 'infinite' or row['Сила'] >= 3:
        row['strong'] = 'Сильный стенд'
    else:
        row['strong'] = 'Слабый стенд'
    return row['strong']

begin_dataset['strong'] = begin_dataset.apply(assign_strongest_stands, axis = 1)



In [318]:
pivot_strong_stands = begin_dataset.value_counts('strong').reset_index().rename(columns = {'count': 'Количество'})

display(pivot_strong_stands)

strong_stands_pie = px.pie(pivot_strong_stands, names='strong', values='Количество', width = 500, height = 500, hole = 0.5, color_discrete_sequence=['#db2724', '#24d8db'])
strong_stands_pie.update_layout(title = {'text': 'Распределение стендов по силе', 'x': 0.5}, showlegend = False)
strong_stands_pie.show()




,strong,Количество
0,Сильный стенд,97
1,Слабый стенд,59


Все равно по распределению видно, что у нас **больше именно сильных стендов**

### 2.3.2 -- Скорость

Теперь посмотрим, что там по скорости стендов

In [319]:
speed_stats = begin_dataset['Скорость'].value_counts()

display(speed_stats)

Скорость
4            41
3            35
5            29
1            27
2            14
undefined     6
infinite      3
unknown       1
Name: count, dtype: int64

Как видно, у нас почти очень быстрые стенды получились, причем медленных прямо мало

Посмотрим на стенды underfined и unknown

In [320]:
display(begin_dataset[(begin_dataset['Скорость'] == 'undefined') | (begin_dataset['Скорость'] == 'unknown')])

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
96,Sky High,4,undefined,4,3,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
99,Bohemian Rhapsody,4,undefined,infinite,5,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
100,Burning Down the House,4,undefined,undefined,undefined,undefined,undefined,Part 6: Stone Ocean,Сильный стенд
102,Dragon's Dream,2,undefined,undefined,5,undefined,undefined,Part 6: Stone Ocean,Слабый стенд
104,"Green, Green Grass of Home",3,unknown,unknown,unknown,unknown,unknown,Part 6: Stone Ocean,Сильный стенд
141,California King Bed,5,undefined,1,4,1,1,Part 8: JoJolion,Сильный стенд
144,Paisley Park,3,undefined,5,5,2,3,Part 8: JoJolion,Сильный стенд


Поступим с такими значениями скорости точно также, как и с силой, просто распределим остальные параеметры на основе имеющихся

In [321]:
stands_speed_stats = begin_dataset[
    (begin_dataset['Скорость'] != 'undefined') & (begin_dataset['Скорость'] != 'unknown') & (begin_dataset['Скорость'] != 'infinite')
]

display(stands_speed_stats)



,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд
...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд


Построим график распределения

In [322]:
pivot_speed_stands = stands_speed_stats.value_counts('Скорость').reset_index()

display(pivot_speed_stands)

stands_speed_histogram = px.histogram(pivot_speed_stands, x = 'Скорость', y = 'count', nbins = 8)

stands_speed_histogram.show()

,Скорость,count
0,4,41
1,3,35
2,5,29
3,1,27
4,2,14


In [323]:
sum_speed = pivot_speed_stands['count'].sum()

pivot_speed_stands['probability'] = pivot_speed_stands['count'] / sum_speed

display(pivot_speed_stands)

,Скорость,count,probability
0,4,41,0.280822
1,3,35,0.239726
2,5,29,0.198630
3,1,27,0.184932
4,2,14,0.095890


In [327]:
stands_speed_to_replace = begin_dataset[(begin_dataset['Скорость'] == 'undefined') | (begin_dataset['Скорость'] == 'unknown')]

display(stands_speed_to_replace)

def replace_speed(row):
    if row['Скорость'] == 'undefined' or row['Скорость'] == 'unknown':
        row['Скорость'] = np.random.choice(pivot_speed_stands['Скорость'], p = pivot_speed_stands['probability'])
    return row['Скорость']

stands_speed_to_replace['Скорость'] = stands_speed_to_replace.apply(replace_speed, axis = 1)

begin_dataset.loc[stands_speed_to_replace.index, 'Скорость'] = stands_speed_to_replace['Скорость']

display(begin_dataset)

display(begin_dataset.value_counts('Скорость').reset_index())

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong


,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
0,Anubis,4,4,1,5,1,3,Part 3: Stardust Crusaders,Сильный стенд
1,Atum,2,3,2,4,2,2,Part 3: Stardust Crusaders,Слабый стенд
2,Bastet,1,1,4,5,1,1,Part 3: Stardust Crusaders,Слабый стенд
3,Cream,4,4,2,3,3,2,Part 3: Stardust Crusaders,Сильный стенд
4,Death Thirteen,3,3,1,4,2,4,Part 3: Stardust Crusaders,Сильный стенд
...,...,...,...,...,...,...,...,...,...
151,Nightbird Flying,1,5,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
152,Purple Haze Distortion,5,4,1,1,3,unknown,Light Novel: Purple Haze Feedback,Сильный стенд
153,Rainy Day Dream Away,1,4,5,5,1,1,Light Novel: Purple Haze Feedback,Слабый стенд
154,Remote Romance,1,2,infinite,4,3,1,Featured,Слабый стенд


,Скорость,count
0,4,44
1,3,37
2,1,29
3,5,29
4,2,14
5,infinite,3


У нас появились 3 стенда с бесконечной скоростью, интересно, посмотрим на них

In [329]:
infinite_speed_stands = begin_dataset[
    begin_dataset['Скорость'] == 'infinite'
]

display(infinite_speed_stands)

,Stand,Сила,Скорость,Дальность,Выносливость,Точность,Потенциал,Story,strong
77,Gold Experience Requiem,infinite,infinite,infinite,infinite,infinite,infinite,Part 5: Vento Aureo,Сильный стенд
83,Notorious B.I.G,5,infinite,infinite,infinite,1,5,Part 5: Vento Aureo,Сильный стенд
109,Made in Heaven,4,infinite,3,5,3,5,Part 6: Stone Ocean,Сильный стенд


Тут у нас уже знакомый GER (Gold Experience Requiem)

Также у нас здесь есть **Notorius B.I.G**, да, это поистине устрашающий стенд, несмотря на то, что его обладатель умер, стенд продолжает преследовать все вокруг, подстраивается под пространство и поглощает абсолютно все. Скорость зависит от того, кого он преслудует. будет преследовать улитку -- будет двигаться со скоростью чуть большей улитики, будет преследовать самолет -- значит будет двигаться быстрее самолета

<p align="center">
    <img src="https://i.ytimg.com/vi/qvk0zrIqm4w/maxresdefault.jpg" alt="Да... Страшный стенд" height = "200", width="400">
</p>

Про **Made in Heaven**, к сожалению, я не знаю ничего, так как в самом разгаре просмотра Stone Ocean, но все равно приложу фотку стенда, ~~будет мне спойлером :)~~

<p align="center">
    <img src="https://static.wikia.nocookie.net/zlodei/images/b/bb/Made_in_Heaven.webp/revision/latest/scale-to-width-down/562?cb=20240217153358&path-prefix=ru" alt="Да... Страшный стенд" height = "400", width="200">
</p>

Да, неплохая лошадка, выглядит мощной, раз я там видел Пуччи, то думаю, это его реквием, может быть 🤔


### 2.3.3 -- Дальность действия

# Выводы

Большинство стендов обладают огромной силой

Самый сильный стенд всех времен и народов -- GER (Gold Experience Requiem)